In [ ]:
#! pip install -U pip setuptools wheel
#! pip install -U 'spacy[cuda12x]'
#! python -m spacy download en_core_web_trf
#! python -m spacy download en_core_web_sm
#! pip install pandas

Codigo que remove pontuacoes, remove stop_words, remove ouliers dos dados do scraper

salva um csv 


### Baixar CSV DAtaset

Para realizar o tratamento de dados a seguir, o IMDB Dataset deve ser baixado, e colocado na pasta raiz do projeto




In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

import spacy
from spacy.tokens import Doc
from tqdm import tqdm

In [2]:
# Carregar o modelo em inglês
nlp = spacy.load("en_core_web_sm", exclude=["parser", "ner"])

FILE_PATH = "D:/tcc2/guilherme/3_Data_Processing/data/scrapper_reviews_english_original.csv" 

df = pd.read_csv(FILE_PATH)
df = df[['review','rating']]
df

,review,rating
0,This movie takes place off in fantasy land som...,1
1,"Too sentimental, too pathetic, too slow, too c...",1
2,There are some things in this life we will nev...,1
3,This movie was inspired by the brilliant Stir ...,1
4,"For the life of me, I can't understand all the...",1
...,...,...
44422,"Uma Thurman returns as the Bride, who this tim...",10
44423,"Quentin Tarantino's ""Kill Bill: Vol. 2"" comple...",10
44424,Vol 2 is completely different tone then the fi...,10
44425,This movie is great in that it mixes together ...,10


In [3]:
df.isnull().sum()

review    0
rating    0
dtype: int64

In [4]:
df.duplicated().sum()

16

In [5]:
# Remove linhas de reviews_treated duplicadoss
df = df.drop_duplicates()

In [6]:
df.duplicated().sum()

0

In [8]:
from transformers import BertTokenizer

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def get_qtd_tokens(x, tokenizer):
    tokens = bert_tokenizer.tokenize(x)
    return len(tokens)



In [ ]:
# Calcular o tamanho dos textos
qtd_tokens = df['review'].apply(lambda x: get_qtd_tokens(x, bert_tokenizer))
df = df.assign(qtd_tokens =qtd_tokens )
df


In [ ]:
df['qtd_tokens'].describe()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde

# Plotar o gráfico
def plot_graf_qnt_tokens(series_list):
    
    mean_length = series_list.mean()
    
    plt.figure(figsize=(10, 6))
    plt.hist(series_list, edgecolor='k', alpha=0.7, density=True)  # Adicione o argumento density=True
    plt.axvline(mean_length, color='r', linestyle='dashed', linewidth=1)
    plt.text(mean_length*1.1, plt.ylim()[1]*0.9, f'Média: {mean_length:.2f}', color='r')

    # # Adicionar a linha de densidade
    # density = gaussian_kde(df['qtd_tokens'])
    # xs = np.linspace(df['qtd_tokens'].min(), df['qtd_tokens'].max(), 200)
    # plt.plot(xs, density(xs), color='blue')

    plt.title('Distribuição do Tamanho dos Textos')
    plt.xlabel('Quantidade de Tokens')
    plt.ylabel('Frequência')
    plt.show()

plot_graf_qnt_tokens(df['qtd_tokens'])

### Remocao de outliers

In [ ]:
def remove_outliers(df, column):
    # Calcular o IQR
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    print(f"Q1: {Q1}")
    print(f"Q3: {Q3}")
    print(f"IQR: {IQR}")

    # Definir os limites para outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    print(f"Limite inferior: {lower_bound}")
    print(f"Limite superior: {upper_bound}")

    # Remover outliers
    df_without_outliers = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    print(f"Número de outliers removidos: {len(df) - len(df_without_outliers)}")

    return df_without_outliers


print("Df com outliers: ", df["qtd_tokens"].size)
df_without_outliers = remove_outliers(df, "qtd_tokens")
print("Df sem outliers: ", df_without_outliers["review"].size)
df_without_outliers["qtd_tokens"].describe()

In [ ]:
plot_graf_qnt_tokens(df_without_outliers['qtd_tokens'])

In [ ]:
print("Max value in df sem outliers:")
df['qtd_tokens'].describe()


In [ ]:
print("Max value in df com outliers:")
df_without_outliers.describe()


In [ ]:
df = df_without_outliers

In [ ]:
DATA_PATH = f"D:/tcc2/guilherme/3_Data_Processing/data/Scraper_Dataset_english_sem_outliers.csv"
df.to_csv(DATA_PATH, index=False)

# Funcoes para pre-processamento


In [ ]:
# O tokenizador do Bert já faz isso
def remove_extra_spaces(text):
    r"""Remover espacos, quebra de linhas e tabulações do inicio
    e fim de frases e ubstituir sequências de espaços
    por um espaço"""

    return re.sub(r"\s+", " ", text).strip()



# Não usar nos dados do scraper
def remove_tags(raw_text):
    r"""Remove tags HTML <.*?>, mas pode remover texto destacado."""
    cleaned_text = re.sub(re.compile("<.*?>"), "", raw_text)

    return cleaned_text


# O tokenizer bert-base-uncased faz automaticamente
def normalize_lowercase(text: str) -> str:
    r"""Converte todas as palavras para forma minuscula"""
    return text.lower()


def remove_stopwords_and_puntuaction(doc: Doc | str):
    r"""Remove pontuacoes,remove stop_words,
    e retorna so o texto"""
    if not isinstance(doc, Doc):

        doc = nlp(doc)

    return " ".join(
        [token.text for token in doc if not token.is_stop and not token.is_punct]
    )

# funcoes de tratamento spacy
def lemmatize_text_remove_stopwords_and_puntuaction(doc: Doc | str):
    r"""Reduz a palavra a lemma, remove pontuacoes,
    remove stop_words"""
    if not isinstance(doc, Doc):

        doc = nlp(doc)

    return " ".join(
        [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    )



# funcoes ilustrativas, são muito ineficientes para serem usadas sozinhas
# def remove_punctuation(doc):
#     return " ".join([token.text for token in doc if not token.is_punct])



# def remove_stopwords(doc):
#     return " ".join([token.text for token in doc if not token.is_stop])



# def lemmatize_text(doc):
#     return " ".join([token.lemma_ for token in doc])

In [ ]:
from typing import List, Literal
from tqdm import tqdm


def execute_all_trataments(
    series_review: pd.Series, FINAL_SPACY_FORMAT: Literal["lemma", "text"]
) -> List[str]:
    if FINAL_SPACY_FORMAT not in ["lemma", "text"]:
        raise ValueError("FINAL_SPACY_FORMAT != [lemma | text]")

    reviews_treated = series_review

    # reviews_treated = reviews_treated.apply(remove_tags)
    ### reviews_treated = reviews_treated.apply(normalize_lowercase)
    reviews_treated = reviews_treated.apply(remove_extra_spaces)

    len_texts = len(reviews_treated)

    docs = list(
        tqdm(
            nlp.pipe(
                reviews_treated,
                batch_size=1000,
                disable=["parser", "ner"],
            ),
            colour="green",
            desc="Progresso: ",
            total=len_texts,
        )
    )

    if FINAL_SPACY_FORMAT == "lemma":

        reviews_treated = [
            lemmatize_text_remove_stopwords_and_puntuaction(doc) for doc in docs
        ]

    elif FINAL_SPACY_FORMAT == "text":

        reviews_treated = [remove_stopwords_and_puntuaction(doc) for doc in docs]

    reviews_treated = [remove_extra_spaces(review) for review in reviews_treated]

    return reviews_treated

In [ ]:
# Aplicar as funções de str ao DataFrame
# definir como True para saída em forma de token.lemma, ou False para token.text
FINAL_SPACY_FORMAT= "text" 
reviews_treated= execute_all_trataments(df["review"],FINAL_SPACY_FORMAT)

In [ ]:
reviews_treated

In [ ]:
df["review"]= reviews_treated


In [ ]:
df.duplicated(subset=["review", "rating"]).sum()


In [ ]:
df = df.drop_duplicates(subset=["review", "rating"], ignore_index=True)

In [ ]:
df

In [ ]:
df=df.drop(columns=['qtd_tokens'])


In [ ]:
df

In [ ]:
# Calcular o tamanho dos textos
qtd_tokens = df['review'].apply(lambda x: get_qtd_tokens(x, bert_tokenizer))


In [ ]:
df = df.assign(qtd_tokens=qtd_tokens) 
df 


In [ ]:
df['qtd_tokens'].describe()


In [ ]:
plot_graf_qnt_tokens(df['qtd_tokens'])

In [ ]:
df_text_sem_outliers = remove_outliers(df, "qtd_tokens")
df_text_sem_outliers

In [ ]:
plot_graf_qnt_tokens(df_text_sem_outliers['qtd_tokens'])

In [ ]:
df_text_sem_outliers.describe()

In [ ]:
df = df_text_sem_outliers

In [ ]:
FINAL_SPACY_FORMAT

In [ ]:
DATA_PATH = f"D:/tcc2/guilherme/3_Data_Processing/data/Scraper_Dataset_treated_{FINAL_SPACY_FORMAT}_sem_outliers.csv"
df.to_csv(DATA_PATH, index=False)

## Vizualizacao dos dados após tratamento

carregar csv já tratado


In [ ]:
import pandas as pd


DATA_PATH = f"D:/tcc2/guilherme/3_Data_Processing/data/Scraper_Dataset_treated_text_sem_outliers.csv"
df = pd.read_csv(DATA_PATH)

In [ ]:
# import collections

# # Concatenar todas as avaliações em uma única string
# all_reviews = ' '.join(df['review'])

# # Dividir a string em palavras
# words = all_reviews.split()

# # Contar a frequência de cada palavra
# word_counts = collections.Counter(words)

# # Exibir as 10 palavras mais comuns
# for word, count in word_counts.most_common(-10):
#     print(f'Palavra: {word}, Frequência: {count}')

# len(word_counts)

In [ ]:
# import numpy as np

# # Calcular estatísticas básicas
# frequencies = list(word_counts.values())
# mean_freq = np.mean(frequencies)
# median_freq = np.median(frequencies)
# min_freq = np.min(frequencies)
# max_freq = np.max(frequencies)
# std_freq = np.std(frequencies)

# print(f"Média de frequência: {mean_freq}")
# print(f"Mediana de frequência: {median_freq}")
# print(f"Frequência mínima: {min_freq}")
# print(f"Frequência máxima: {max_freq}")
# print(f"Desvio padrão de frequência: {std_freq}")

# # Calcular o IQR
# Q1 = np.percentile(frequencies, 25)
# Q3 = np.percentile(frequencies, 75)
# IQR = Q3 - Q1

# # Identificar outliers
# outliers = [freq for freq in frequencies if freq < Q1 - 1.5 * IQR or freq > Q3 + 1.5 * IQR]

# print(f"Número de outliers: {len(outliers)}")


In [ ]:
# import numpy as np


# # Calcular o tamanho dos textos
# df['qtd_tokens'] = df['review'].apply(len)

# # Calcular a média e o desvio padrão
# mean_length = df['qtd_tokens'].mean()
# std_length = df['qtd_tokens'].std()

# # Definir o limite para remoção de outliers (por exemplo, 3 desvios padrão)
# threshold_min = mean_length - 3 * std_length
# threshold_max = mean_length + 3 * std_length

# # Filtrar os textos dentro do limite
# df_filtered = df[(df['qtd_tokens'] >= threshold_min) & (df['qtd_tokens'] <= threshold_max)]

# # Calcular a nova média
# mean_length_filtered = df_filtered['qtd_tokens'].mean()

# # Plotar o gráfico sem outliers
# plt.figure(figsize=(10, 6))
# plt.hist(df_filtered['qtd_tokens'], bins=30, edgecolor='k', alpha=0.7)
# plt.axvline(mean_length_filtered, color='r', linestyle='dashed', linewidth=1)
# plt.text(mean_length_filtered*1.1, plt.ylim()[1]*0.9, f'Média: {mean_length_filtered:.2f}', color='r')
# plt.title('Distribuição do Tamanho dos Textos (Sem Outliers)')
# plt.xlabel('Tamanho do Texto')
# plt.ylabel('Frequência')
# plt.show()

# result_string = (
#     f"Distribuição do Tamanho dos Textos (Sem Outliers)\n"
#     f"Média do tamanho dos textos: {mean_length_filtered:.2f}\n"
#     f"Limite inferior para remoção de outliers: {threshold_min:.2f}\n"
#     f"Limite superior para remoção de outliers: {threshold_max:.2f}\n"
#     f"Total de textos considerados: {len(df_filtered)}"
# )

# print(result_string)

Aqui está um exemplo ajustado usando quantis para remover outliers: